In [1]:
import csv
import requests
import pandas as pd
from io import StringIO

Data source:
- Bureau of Reclamation: Lake Mead, Lake Mohave, Lake Havasu
https://www.usbr.gov/uc/water/hydrodata/reservoir_data/920/csv/17.csv
- United States Geological Survey: San Carlos, , Bartlett Dam

- US Army Corps of Engineers (only go to previous 180 days): Alamo, Painted Rock 
- Salt River Project (only go to previous 1 year): Theodore Roosevelt, Horse Mesa Dam

In [2]:
LOWER_RESERVOIR_RECLAMATION = {921:['lake mead', 'AZ, NV'], 922: ['lake mohave', 'AZ, NV'], 923: ['lake havasu', 'AZ, CA']}
reclamation_url_head = 'https://www.usbr.gov/uc/water/hydrodata/reservoir_data/'
reclamation_url_tail = '/csv/17.csv'

In [3]:
def read_csv_from_url(url):
    response = requests.get(url)
    assert response.status_code == 200, 'Failed to download file'
    data = response.content.decode('utf-8')
    return pd.read_csv(StringIO(data))

In [8]:
columns = ['datetime', 'name', 'state', 'storage']
df = pd.DataFrame(columns=columns)

In [7]:
for reservoir_code, info in LOWER_RESERVOIR_RECLAMATION.items():
    # print(info)
    data = read_csv_from_url(reclamation_url_head + str(reservoir_code) + reclamation_url_tail)
    data['name'] = info[0]
    data['state'] = info[1]
    data['datetime'] = pd.to_datetime(data['datetime'])
    data.set_index('datetime',inplace=True)
    data = data.groupby('name')['storage'].resample('M').mean().reset_index()
    df = pd.concat([df, data], ignore_index=True)

KeyError: 'datetime'

In [24]:
data.dtypes

datetime     object
storage     float64
name         object
state        object
dtype: object

In [17]:
info = ['lake mead', 'AZ, NV']
data = read_csv_from_url(reclamation_url_head + str(921) + reclamation_url_tail)
data['name'] = info[0]
data['state'] = info[1]
data['datetime'] = pd.to_datetime(data['datetime'])
data = data.groupby('name')['storage'].resample('M').mean().reset_index()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [16]:
data

,storage,name,state
datetime,,,
1937-05-28,10046048.3,lake mead,"AZ, NV"
1937-05-29,10138665.0,lake mead,"AZ, NV"
1937-05-30,10228958.7,lake mead,"AZ, NV"
1937-05-31,10302056.6,lake mead,"AZ, NV"
1937-06-01,10373112.5,lake mead,"AZ, NV"
...,...,...,...
2023-07-13,8295460.0,lake mead,"AZ, NV"
2023-07-14,8310501.8,lake mead,"AZ, NV"
2023-07-15,8326331.6,lake mead,"AZ, NV"


In [15]:
data.set_index('datetime',inplace=True)

In [ ]:
d